<a href="https://colab.research.google.com/github/khadeejaazizi/AER850project3/blob/main/AER850_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image_path = "/content/drive/My Drive/project 3 data/motherboard_image.JPEG"
img_real = cv2.imread(image_path, cv2.IMREAD_COLOR)
img_real = cv2.rotate(img_real, cv2.ROTATE_90_CLOCKWISE)

img = cv2.imread(image_path, cv2.IMREAD_COLOR)
img = cv2.GaussianBlur(img, (47, 47), 4)
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img_gray = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 7)
img_gray = cv2.rotate(img_gray, cv2.ROTATE_90_CLOCKWISE)

# Edge detection using Canny
edges = cv2.Canny(img_gray, threshold1=50, threshold2=300)
edges = cv2.dilate(edges, None, iterations=10)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank mask
mask = np.zeros_like(img_real)

# Draw the largest contour on the mask
cv2.drawContours(image=mask, contours=[max(contours, key=cv2.contourArea)], contourIdx=-1, color=(255, 255, 255), thickness=cv2.FILLED)

# Extract PCB using bitwise_and
masked_img = cv2.bitwise_and(mask, img_real)

# Display each result separately
# Original image
plt.figure(figsize=(8, 6))
plt.title("Original Image")
plt.imshow(cv2.cvtColor(img_real, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Edge detection
plt.figure(figsize=(8, 6))
plt.title("Edge Detection")
plt.imshow(edges, cmap="gray")
plt.axis("off")
plt.show()

# Mask image
plt.figure(figsize=(8, 6))
plt.title("Mask Image")
plt.imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Final extracted PCB
plt.figure(figsize=(8, 6))
plt.title("Extracted PCB")
plt.imshow(cv2.cvtColor(masked_img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()


# New Section

In [ ]:
# Import required libraries
from ultralytics import YOLO
import os

from google.colab import drive
drive.mount('/content/drive')

# Define paths to dataset and model directory
data_path = "/content/drive/My Drive/project 3 data/data/data.yaml"
model_save_path = "/content/drive/My Drive/project 3 data/data"

# Load YOLOv8 Nano model (pretrained)
model = YOLO('yolov8n.pt')

# Set training hyperparameters
hyperparameters = {
    'epochs': 150,      # Number of training epochs
    'batch': 4,        # Batch size per epoch
    'imgsz': 960,       # Image size (minimum 900 for small components)
    'name': 'pcb_detector',  # Name of the model
    'data': data_path,  # Path to dataset configuration file
    'project': model_save_path  # Directory to save results
}

# Train the YOLOv8 model
print("Starting training...")
model.train(**hyperparameters)

# Notify when training is complete
print(f"Training complete. Model saved at: {os.path.join(model_save_path, 'pcb_detector')}.")

In [30]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

# Path to the trained YOLOv8 model
model_path = "/content/drive/My Drive/project 3 data/data/pcb_detector3/weights/best.pt"

# Load the trained YOLOv8 model
model = YOLO(model_path, task="detect")

# Paths to evaluation images
eval_image_paths = [
    "/content/drive/My Drive/project 3 data/data/evaluation/rasppi.jpg",
    "/content/drive/My Drive/project 3 data/data/evaluation/arduno.jpg",
    "/content/drive/My Drive/project 3 data/data/evaluation/ardmega.jpg"
]


# Directory to save evaluation results
output_dir = "/content/drive/My Drive/project 3 data/data/labelz"
os.makedirs(output_dir, exist_ok=True)

# Define a color palette for different component types
def get_unique_color(class_id):
    colors = [
        (0, 255, 0),  # Green
        (0, 0, 255),  # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow

    ]
    return colors[class_id % len(colors)]  # Cycle through colors if classes exceed the palette

# Function to evaluate the model and save annotated results
def evaluate_and_save_results(model, image_paths):
    saved_image_paths = []
    for image_path in image_paths:
        # Load the image
        image = cv2.imread(image_path)

        # Normalize resolution to ensure consistency
        target_width = 950  # Define a fixed width for all images
        aspect_ratio = image.shape[1] / image.shape[0]
        target_height = int(target_width / aspect_ratio)
        image = cv2.resize(image, (target_width, target_height))


        # Run prediction
        results = model.predict(image, save=False, imgsz=928, conf=0.35)
        boxes = results[0].boxes  # Detection boxes
        names = results[0].names  # Class names

        # Draw bounding boxes and labels manually
        for box in boxes:
            # Extract box coordinates, confidence, and class ID
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box corners
            conf = box.conf[0]  # Confidence
            class_id = int(box.cls[0])  # Class ID

            # unique color for each component
            color = get_unique_color(class_id)
            label = f"{names[class_id]} {conf:.2f}"

            # Define custom styling
            thickness = 2  # Thin line
            font = cv2.FONT_HERSHEY_COMPLEX
            font_scale = 0.33  # Smaller font size
            font_thickness = 1

            # Draw bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

            # label text
            label_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
            label_x, label_y = x1, y1 - 10
            label_y = max(label_y, 10)  # Prevent label from going off the top
            cv2.rectangle(image, (label_x, label_y - label_size[1] - 2), (label_x + label_size[0], label_y + 2), color, -1)
            cv2.putText(image, label, (label_x, label_y), font, font_scale, (0, 0, 0), font_thickness) #tried white and black text
        # Save the annotated image
        filename = os.path.basename(image_path)
        save_path = os.path.join(output_dir, filename)
        cv2.imwrite(save_path, image)
        saved_image_paths.append(save_path)
        print(f"Saved evaluation result: {save_path}")
    return saved_image_paths


# Evaluate and save results
evaluation_results = evaluate_and_save_results(model, eval_image_paths)



0: 640x928 6 Capacitors, 9 Connectors, 1 Diode, 1 Electrolytic Capacitor, 7 ICs, 1 Led, 14 Resistors, 1 Transistor, 316.5ms
Speed: 7.4ms preprocess, 316.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 928)
Saved evaluation result: /content/drive/My Drive/project 3 data/data/labelz/rasppi.jpg

0: 640x928 1 Button, 9 Capacitors, 7 Connectors, 2 Electrolytic Capacitors, 7 ICs, 4 Leds, 15 Resistors, 299.9ms
Speed: 7.4ms preprocess, 299.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 928)
Saved evaluation result: /content/drive/My Drive/project 3 data/data/labelz/arduno.jpg

0: 832x928 1 Button, 8 Capacitors, 8 Connectors, 2 Electrolytic Capacitors, 7 ICs, 17 Resistors, 393.5ms
Speed: 10.0ms preprocess, 393.5ms inference, 1.7ms postprocess per image at shape (1, 3, 832, 928)
Saved evaluation result: /content/drive/My Drive/project 3 data/data/labelz/ardmega.jpg
